In [1]:
# импортируем все нужные функции 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

Поиск неплохого решения занял много времени. В ходе работы оказалось, что стемминг, леммитизация, а так же исключение знаков препинания плохо влияют на качество классификации, по крайней accuracy. Так же оказалось, что примитивная модель может быть намного лучше сложных и страшных и что логистическая регрессия лучший классификатор для данной задачи. Модели основанные символьных нграммах сложны и при этом работают хуже, поэтому здесь также не рассматриваются

В данном ноутбуке не показано, как находился оптимальный алгоритм классификации (делал это на данной выборке + несколько выборок из nltk), показан только поиск гиперпараметров для выбранных методов преобразования и классификации текста

In [8]:
# сначала считаем данные
data_train = pd.read_csv('products_sentiment_train .tsv', sep='\t', header=None)
data_test = pd.read_csv('products_sentiment_test .tsv', sep='\t', index_col='Id')
labels = data_train.iloc[:,1]
data_train.columns = ['text', 'lab']

In [9]:
data_train.head()

,text,lab
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [10]:
# создадим pipeline из векторизатора и классификатора
pl = Pipeline([('vectorizer', CountVectorizer()),('classifier', LogisticRegression())])

In [11]:
#создадим сетку разумных для модели параметров
from sklearn import model_selection
parameters_grid = {
    'vectorizer__ngram_range' : [(1,2),(1,3),(1,4)],
    'vectorizer__stop_words': [None, 'english'],
    'vectorizer__min_df' : [1,3,5],
    'classifier__class_weight': [None, 'balanced']
}
grid_cv = model_selection.GridSearchCV(pl, parameters_grid, scoring = 'accuracy', cv = 4)

In [12]:
#запускаем полный поиск по сетке (2 рандомных поиска до этого не увенчались успехом, поэтому полный)
grid_cv.fit(data_train.text, labels)
print grid_cv.best_score_
print grid_cv.best_params_

0.7685
{'vectorizer__min_df': 1, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 2), 'vectorizer__stop_words': None}


In [13]:
#смотрим на параметры и добавляем tfidf, с ним оказалось на 2 ошибки меньше)
from sklearn.feature_extraction.text import TfidfTransformer
cv = CountVectorizer(ngram_range=(1,2), min_df=1)
tf =  TfidfTransformer()
lr =  LogisticRegression(class_weight='balanced', solver='sag')
a = cv.fit_transform(data_train.text)
a1 = cv.transform(data_test.text)
b = tf.fit_transform(a)
b1 = tf.transform(a1)
lr.fit(b, labels)
pre = lr.predict(b1)
lab = range(len(pre))

In [14]:
#записываем результат 
d = {'id': lab, 'y': pre}
data1 = pd.DataFrame(data=d)
data1.to_csv('otvety1.csv', index=False)

In [15]:
#проведем нормализацию и стеммизацию текста. еще раз проведем поиск и сравним качество
import nltk
from nltk.stem import SnowballStemmer
stop_symbols = '.,:;"-\n\r'
stop_words= nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer('english')
# работаем с начальной выборкой
good_texts = []
for text in data_train.text:
    v=([stemmer.stem(x) for x in [y.strip(stop_symbols) for y in text.lower().split()] if x and (x not in stop_words)])
    d = ' '
    for word in v:
        d = d+word+" "
    good_texts.append(d)
# теперь с тестовой
good_texts_test = []
for text in data_test.iloc[:,0]:
    v=([stemmer.stem(x) for x in [y.strip(stop_symbols) for y in text.lower().split()] if x and (x not in stop_words)])
    d = ''
    for word in v:
        d = d+word+" "
    good_texts_test.append(d)    

In [21]:
#создаем pipeline сразу с трансформером, сетка параметров остается такой же
pl1 = Pipeline([('vectorizer', CountVectorizer()),('transformer',TfidfTransformer()),('classifier', LogisticRegression())])
grid_cv1 = model_selection.GridSearchCV(pl1, parameters_grid, scoring = 'accuracy', cv = 4)
grid_cv1.fit(good_texts, labels)
print grid_cv1.best_score_
print grid_cv1.best_params_

0.7595
{'vectorizer__min_df': 1, 'classifier__class_weight': 'balanced', 'vectorizer__ngram_range': (1, 3), 'vectorizer__stop_words': None}


Как видно качество ухудщилось по сравнению с первой моделью без обработки текста, хотя там не было tfidf.

In [25]:
#записываем результат 
pre = grid_cv1.predict(good_texts_test)
lab = range(len(pre))
d = {'id': lab, 'y': pre}
data1 = pd.DataFrame(data=d)
data1.to_csv('otvety1.csv', index=False)

На kaggle такое предсказание оказалось худшим)

Если есть желание обсудить задачу, попробовать вместе найти пути дальнейшего улучшения качества, разобраться с казалось бы простой библиотекой gensim word2vec и doc2vec пишите на почту: andrei322glotov@yandex.ru 
Так же можем обсудить задачи по идентификации и оттоку